# Récupération des données

In [67]:
import pandas as pd
from math import isnan
import time 
import numpy as np

In [3]:
df = pd.read_excel("PS_INFO.xlsx")

In [248]:
def normalisation_bac(i,forme,epreuve):
    ns = []
    add = 0
    for e in range(len(df)):
        if Form[e] == forme:
            if isnan(df[epreuve][e]) == False:
                ns.append(df[epreuve][e])
    ecart_type = np.std(ns)
    moyenne = np.mean(ns)
    normalise = (df[epreuve][i]-moyenne)/ecart_type
    normaliser = (normalise*5)+10
    if normaliser > 20 :
        normaliser =20
    if normaliser < 0:
        normaliser = 0
    return normaliser
    print(normaliser)

In [249]:
normalisation_bac(707,"Scientifique","Note à l'épreuve de Mathématiques")

0

In [251]:
a , b  = recupDBC(df,707)
print(a)

{'Validation': 1, 'Num_Can': 708, 'SEXE': 'M', 'ANGLAIS': '?', 'FILIERE': 'Technologique', 'Bourse': False, 'Nom_Eta': nan, 'Com_Eta': nan, 'Depa_Eta': nan}


In [4]:
FR_Oral = df["Note à l'épreuve de Oral de Français (épreuve anticipée)"]
FR_Ecrit = df["Note à l'épreuve de Ecrit de Français (épreuve anticipée)"]

In [85]:
start = time.time()
for i in range(4600):
    a = recupDBC(df,i)
end = time.time()
elapsed = end - start
print(elapsed)

7.004727840423584


In [63]:
df["Note à l'épreuve de Mathématiques"][4563]

9.0

In [135]:
a , b  = recupDBC(df,4563)
print(b["MATH"]["NOTE"])


NOTE


In [5]:
# Es-ce que le candidats à son Bac de Français ?
def BAC_FR(i):
    if (isnan(FR_Oral[i]) and isnan(FR_Ecrit[i])):
        return "NON"
    else :
        return "OUI"

In [225]:
def recupDBC(df,i):
    #Création de matière qui stock les notes du candidat
    matiere = pd.DataFrame()
    #Création d'une DataFrame pour le Candidats i
    DBC ={
        'Validation':0,
        'Num_Can':df['ID'][i],
        'SEXE':df['Sexe'][i], 
        'ANGLAIS':'?',
        'FILIERE':'?',
        'Bourse': False, 
        'Nom_Eta' : df["Libellé établissement"][i], 
        'Com_Eta' : df["Commune établissement"][i], 
        'Depa_Eta' : df["Département établissement"][i]
    }
    
    
    #Récupération des informations:
    #Quelle est sa formations scolaire précédent sa candidatures
    Form = df['Série diplôme'][i]
    
    
    #Prise en compte des notes de BAC de Français
    FR_Oral = df["Note à l'épreuve de Oral de Français (épreuve anticipée)"][i]
    FR_Ecrit = df["Note à l'épreuve de Ecrit de Français (épreuve anticipée)"][i]   
    
    
    #Le Candidats est-il boursier ?
    if(df["Boursier"][i] in ["Boursier de l'enseignement supérieur", 'Boursier du secondaire']) : 
        DBC['Bourse'] = True # Mise à jour de sa situation de bourse
        
    #Es-ce que le candidats est en formation Générale ou Technologique ?    
        DBC['FILIERE']='Générale'
    else : 
        DBC['FILIERE']='Technologique'
        
    
    #On vas attrivuer des points:
    # -1 vaut une élémination
    #  0 sa candidatures sera prise en compte en manuel
    #  1 sa candidatures vas être 
    if(df['Candidature validée (O/N)'][i]=='NON') : 
        # Si non validé éléminé
        DBC['Validation']= -1    
        
    # On recherche les DAEU Mathématiques.
    # Dans un premier temps nous n'avons que les noms en DAEU
    elif Form == "DAEU" : 
        # On recherche plus précisemment les Daeu-b qui sont les DAEU MAthématiques
        if df["Spécialité diplôme bac Pro et anciens bacs"][i] == "Daeu-b" :
            # Dossier manuel
            DBC['Validation']=0
            DBC['Autre_Form']="Daeu-b"
            return DBC
        else:
            # Si DAEU-a, donc Littérature éléminé
            DBC['Validation']=-1
            
    # On recherche les anciens Bac ou Bac Internationale
    elif(Form in ["Bac général de plus de 15 ans","Baccalauréat International"]) : 
        #Bac en cours mais pas classique => dossier à étudier à la main
        DBC['Validation']=0
        DBC['Autre_Form']="AUTRES"
        return DBC
    
    # On cherche les Bac Scientifiques, qui ont validé leur Bac de français
    elif(BAC_FR(i)=="OUI" and Form=="Scientifique") : 
        DBC["Validation"]=1
        #Mathématiques, les baccalauréat Scientifiques sont des "Anciens Baccalauréat", 
        #et ou on déjà passé leurs baccalauréat
        math = []
        math.append(normalisation_bac(i,"Scientifique","Note à l'épreuve de Mathématiques"))
        matiere['MATH']= math
        
        #Français
        fre = float(normalisation_bac(i,"Scientifique","Note à l'épreuve de Ecrit de Français (épreuve anticipée)"))
        fro = float(normalisation_bac(i,"Scientifique","Note à l'épreuve de Oral de Français (épreuve anticipée)"))
        frt = (fre+fro)/2
        matiere['FRANCAIS']= frt
        
        #Spécialité pour le baccalauréat Scientifique, les notes de Mathématiques Spé , sont incluse avec la note de Math
        # On ne vas rechercher que les notes d'ISN
        SPE=df["Note à l'épreuve de Informatique et Sciences du numérique"][i]
        nsi = normalisation_bac(i,"Scientifique","Note à l'épreuve de Informatique et Sciences du numérique")
        if isnan(SPE) == False  :
            matiere["NSI"]= nsi
            

        
        #Anglais
        LV1=df["LV1"][i]
        LV11=df["LV1.1"][i]
        LV12=df["LV1.2"][i]
        LV2=df["LV2"][i]
        LV21=df["LV2.1"][i]
        LV22=df["LV2.2"][i]
        Choix = ["Note à l'épreuve de Langue vivante 1","Note à l'épreuve de Langue vivante 2"]
        angl=None
        if((LV1=="Anglais" or LV11 =="Anglais") or LV12 =="Anglais"  ): 
            angl="Langue vivante 1"
            agl = normalisation_bac(i,"Scientifique","Note à l'épreuve de Langue vivante 1")
            
        if((LV2=="Anglais" or LV21 =="Anglais") or LV22 =="Anglais"  ): 
            angl="Langue vivante 1"
            agl = normalisation_bac(i,"Scientifique","Note à l'épreuve de Langue vivante 2")

        if(angl!=None) : 
            matiere['ANGLAIS']= agl 
            
    # Recherche des Bac ES
    elif(BAC_FR(i)=="OUI" and Form=="Economique et social" ): 
        DBC["Validation"]=1
        #Mathématiques, les baccalauréat Scientifiques sont des "Anciens Baccalauréat", 
        #et ou on déjà passé leurs baccalauréat
        math = [] 
        math.append(normalisation_bac(i,"Economique et social","Note à l'épreuve de Mathématiques"))
        matiere['MATH']= math
        
        #Français
        fre = float(normalisation_bac(i,"Economique et social","Note à l'épreuve de Ecrit de Français (épreuve anticipée)"))
        fro = float(normalisation_bac(i,"Economique et social","Note à l'épreuve de Oral de Français (épreuve anticipée)"))
        frt = (fre+fro)/2
        matiere['FRANCAIS']= frt
        
        #SES
        ses = normalisation_bac(i,"Economique et social","Note à l'épreuve de Sciences économiques et sociales (SES)")
        matiere['SES']= ses
        
        #Spécialité pour le baccalauréat Scientifique, les notes de Mathématiques Spé , sont incluse avec la note de Math
        # On ne vas rechercher que les notes d'ISN
        SPE=df["Note à l'épreuve de Informatique et Sciences du numérique"][i]
        nsi = normalisation_bac(i,"Economique et social","Note à l'épreuve de Informatique et Sciences du numérique")
        if isnan(SPE) == False  :
            matiere["NSI"]= nsi
        
        #Anglais
        LV1=df["LV1"][i]
        LV11=df["LV1.1"][i]
        LV12=df["LV1.2"][i]
        LV2=df["LV2"][i]
        LV21=df["LV2.1"][i]
        LV22=df["LV2.2"][i]
        Choix = ["Note à l'épreuve de Langue vivante 1","Note à l'épreuve de Langue vivante 2"]
        angl=None
        if((LV1=="Anglais" or LV11 =="Anglais") or LV12 =="Anglais"  ): 
            angl="Langue vivante 1"
            DBC["ANGLAIS"] = "LV1"
            agl = normalisation_bac(i,"Economique et social","Note à l'épreuve de Langue vivante 1")
            
        if((LV2=="Anglais" or LV21 =="Anglais") or LV22 =="Anglais"  ): 
            angl="Langue vivante 2"
            DBC["ANGLAIS"] = "LV2"
            agl = normalisation_bac(i,"Economique et social","Note à l'épreuve de Langue vivante 2")

        if(angl!=None) : 
            matiere['ANGLAIS']= agl 
    # Recherche des candidats STI2D        
    elif(Form=="Sciences et Technologies de l'Industrie et du Développement Durable") : 
        
        DBC['Validation']=1
        
        if isnan(df["Note à l'épreuve de Oral de Français (épreuve anticipée)"]) ==  False:
            matiere['SCIENCES']=
                #Math
            math = [] 
            math.append(normalisation_bac(i,"Sciences et Technologies de l'Industrie et du Développement Durable","Note à l'épreuve de Mathématiques"))
            matiere['MATH']= math

        #Informatique
            matiere['NSI']= 
        
        #Français
            matiere['FRANCAIS']=
            
        #Anglais
            LV1=df["LV1"][i]
            LV2=df["LV2"][i]
            mat=None
            if(LV1=="Anglais") : mat="Langue vivante 1"
            if(LV2=="Anglais") : mat="Langue vivante 2"

            if(mat!=None) : 
                matiere['ANGLAIS']=
                
                
                
                
                
        #Sciences
        matiere['SCIENCES']={
            'NOM' : 'Physique-Chimie et Mathématiques',
            'NOTE' : {
                'P1':'Trimestre 1.1',
                'P2':'Trimestre 2.1',
                'P3':'Trimestre 3.1',
                'T1':'Trimestre 1',
                'T2':'Trimestre 2',
                'T3':'Trimestre 3'
            }
        }

        #Math
        matiere['MATH']={
            'NOM' : 'Mathématiques',
            'NOTE' : {
                'P1':'Trimestre 1.1',
                'P2':'Trimestre 2.1',
                'P3':'Trimestre 3.1',
                'T1':'Trimestre 1',
                'T2':'Trimestre 2',
                'T3':'Trimestre 3'
            }
        }

        #Informatique
        matiere['NSI']={
            'NOM' : "Systemes d''inForm et numerique",
            'NOTE' : {
                'P1':'Trimestre 1.1',
                'P2':'Trimestre 2.1',
                'P3':'Trimestre 3.1',
                'T1':'Trimestre 1',
                'T2':'Trimestre 2',
                'T3':'Trimestre 3'
            }
        }
        
        #Français
        matiere['FRANCAIS']={
            'NOM' : 'Français',
            'NOTE' : {
                'P1':'Trimestre 1.1',
                'P2':'Trimestre 2.1',
                'P3':'Trimestre 3.1',
                'T1':'Trimestre 1',
                'T2':'Trimestre 2',
                'T3':'Trimestre 3'
            }
        }       

        #Anglais
        LV1=df["LV1"][i]
        LV2=df["LV2"][i]
        mat=None
        if(LV1=="Anglais") : mat="Langue vivante 1"
        if(LV2=="Anglais") : mat="Langue vivante 2"

        if(mat!=None) : 
            matiere['ANGLAIS']={
                'NOM' : mat,
                'NOTE' : {
                    'P1':'Trimestre 1.1',
                    'P2':'Trimestre 2.1',
                    'P3':'Trimestre 3.1',
                    'T1':'Trimestre 1',
                    'T2':'Trimestre 2',
                    'T3':'Trimestre 3'
                }
            }  
    #Recherche des candidats de Série Générale (nouveau bac)
    elif(Form=='Série Générale') : 
        
        DBC['Validation']=1

        #Sciences
        matiere['SCIENCES']={
            'NOM' : 'Enseignement scientifique',
            'NOTE' : {
                'P1':'Trimestre 1.1',
                'P2':'Trimestre 2.1',
                'P3':'Trimestre 3.1',
                'T1':'Trimestre 1',
                'T2':'Trimestre 2',
                'T3':'Trimestre 3'
            }
        }

        #Français
        matiere['FRANCAIS']={
            'NOM' : 'Français',
            'NOTE' : {
                'P1':'Trimestre 1.1',
                'P2':'Trimestre 2.1',
                'P3':'Trimestre 3.1',
                'T1':'Trimestre 1',
                'T2':'Trimestre 2',
                'T3':'Trimestre 3'
            }
        }       

        #Anglais
        LV1=df["LV1"][i]
        LV2=df["LV2"][i]
        mat=None
        if(LV1=="Anglais") : mat="Langue vivante 1"
        if(LV2=="Anglais") : mat="Langue vivante 2"

        if(mat!=None) : 
            matiere['ANGLAIS']={
                'NOM' : mat,
                'NOTE' : {
                    'P1':'Trimestre 1.1',
                    'P2':'Trimestre 2.1',
                    'P3':'Trimestre 3.1',
                    'T1':'Trimestre 1',
                    'T2':'Trimestre 2',
                'T3':'Trimestre 3'
                }
            }  

        #Enseignements de spécialités
        mat_interessante={
            "Mathématiques Spécialité":"MATH",
            "Numérique et Sciences Informatiques":"NSI"
        }

        #Les deux enseignements de spécialité de terminale
        mat1 = df["EDS BAC Terminale"][i]
        mat2 = df["EDS BAC Terminale.1"][i]

        for mat in [mat1, mat2] : 
            if(mat in mat_interessante.keys()) :
                matiere[mat_interessante[mat]]={
                    'NOM' : mat,
                    'NOTE' : {
                        'P1':'Trimestre 1.1',
                        'P2':'Trimestre 2.1',
                        'P3':'Trimestre 3.1',
                        'T1':'Trimestre 1',
                        'T2':'Trimestre 2',
                        'T3':'Trimestre 3'
                    }
                }   


        #Option Math complémentaire
        matiere['MATH_COMP']={
            'NOM' : 'Mathématiques Complémentaires',
            'NOTE' : {
                'T1':'Trimestre 1',
                'T2':'Trimestre 2',
                'T3':'Trimestre 3'
            }
        }

        #Option Math expertes
        matiere['MATH_EXP']={
            'NOM' : 'Mathématiques Expertes',
            'NOTE' : {
                'T1':'Trimestre 1',
                'T2':'Trimestre 2',
                'T3':'Trimestre 3'
            }
        }  
        
    #Si le candidats ne rentres dans aucune des conditions  
    else :
        DBC['Validation']=-1
        return DBC  
    return DBC , matiere

Truc à integrer

In [ ]:
for mat in matiere : 
    trimestre=matiere[mat]['NOTE']
    DBC[mat]={}
    for tri in trimestre : 
        DBC[mat][tri] = recupNOTE(df, i, matiere[mat]['NOM'], trimestre[tri], a, b)

In [8]:
def calculnote(DATA, MATRICE, MATIERE):
    res = None
    a = 0
    for mat in MATIERE:
        b = 0
        for x in ['P1', 'P2', 'P3', 'T1', 'T2', 'T3']:
            try:
                DATA[mat][x]
            except:
                continue

            if DATA[mat][x] is not None:
                if res is None:
                    res = DATA[mat][x] * MATRICE[mat][x]
                    b += MATRICE[mat][x]

        if b > 0:
            N = res / b
            if N < MATRICE[mat]['PLANCHE']:
                return "NC"  # Non classé

            res = MATRICE[mat]['COEF'] * N + res
            a = MATRICE[mat]['COEF'] + a

        # Si le candidat n'a pas de note de français ou anglais il sera non classé
        if DATA[mat]['P1'] is None and mat in ['FRANCAIS', 'ANGLAIS']:
            return "NC"

    if res is not None and a != 0:
        res = res / a
    return res


In [120]:
ENS_GEN=['MATH', 'MATH_EXP', 'MATH_COMP', 'SCIENCES', 'NSI', 'SES', 'FRANCAIS', 'ANGLAIS']
MATRICE_G = pd.DataFrame(
    {
        'MATH': [1,2,3,2,3,4,13,8], 
        'MATH_EXP': [1,2,3,2,3,4,6,6], 
        'MATH_COMP': [0,0,0,1,2,3,9,7], 
        'SCIENCES': [1,2,3,2,3,4,4,8], 
        'NSI': [1,1,1,2,2,2,6,8],  
        'SES': [1,1,1,2,2,2,5,7], 
        'ANGLAIS': [1,1,1,1,1,1,4,8], 
        'FRANCAIS': [1,1,1,1,1,1,9,8]
    },
    index = ['P1', 'P2', 'P3', 'T1', 'T2', 'T3', 'COEF', 'PLANCHE']
)

In [121]:
calculnote(recupDBC(df,213),MATRICE_G,ENS_GEN)

TypeError: tuple indices must be integers or slices, not str